# Welcome to the Profiling Tool for the RAPIDS Accelerator for Apache Spark
To run the tool, you need to enter a log path that represents the DBFS location for your Spark GPU event logs.  Then you can select "Run all" to execute the notebook.  After the notebook completes, you will see various output tables show up below.

## GPU Job Tuning Recommendations
This has general suggestions for tuning your applications to run optimally on GPUs.

## Per-Job Profile
The profiler output includes information about the application, data sources, executors, SQL stages, Spark properties, and key application metrics at the job and stage levels.

In [0]:
import json
import requests
import base64
import shlex
import subprocess
import pandas as pd

TOOL_JAR_URL = 'https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark-tools_2.12/22.10.0/rapids-4-spark-tools_2.12-22.10.0.jar'
TOOL_JAR_LOCAL_PATH = '/tmp/rapids-4-spark-tools.jar'

# Profiling tool output directory.
OUTPUT_DIR = '/tmp' 

response = requests.get(TOOL_JAR_URL)
open(TOOL_JAR_LOCAL_PATH, "wb").write(response.content)

Out[1]: 1985484

In [0]:
dbutils.widgets.text("log_path", "")

In [0]:
eventlog_string=dbutils.widgets.get("log_path") 

q_command_string="java -Xmx10g -cp /tmp/rapids-4-spark-tools.jar:/databricks/jars/* com.nvidia.spark.rapids.tool.profiling.ProfileMain --csv --auto-tuner -o {} ".format(OUTPUT_DIR) + eventlog_string
args = shlex.split(q_command_string)
cmd_out = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if cmd_out.returncode != 0:
  dbutils.notebook.exit("Profiling Tool failed with stderr:" + cmd_out.stderr)

In [0]:
import os

app_df = pd.DataFrame(columns = ['appId', 'appName'])

for x in os.scandir(OUTPUT_DIR + "/rapids_4_spark_profile/"):
  tmp_df = pd.read_csv(x.path + "/application_information.csv")
  app_df = app_df.append(tmp_df[['appId', 'appName']])

## GPU Job Tuning Recommendations

In [0]:
app_list = app_df["appId"].tolist()
app_recommendations = pd.DataFrame(columns=['app', 'recommendations'])

for app in app_list:
  app_file = open(OUTPUT_DIR + "/rapids_4_spark_profile/" + app + "/profile.log")
  recommendations_start = 0
  recommendations_str = ""
  for line in app_file:
    if recommendations_start == 1:
      recommendations_str = recommendations_str + line
    if "### D. Recommended Configuration ###" in line:
      recommendations_start = 1
  app_recommendations = app_recommendations.append({'app': app, 'recommendations': recommendations_str}, ignore_index=True)
    
display(app_recommendations)

app,recommendations
app-20220210005817-0212,"Cannot recommend properties. See Comments. Comments: - java.io.FileNotFoundException: File worker_info.yaml does not exist - 'spark.executor.memory' should be set to at least 2GB/core. - 'spark.executor.instances' should be set to (gpuCount * numWorkers). - 'spark.task.resource.gpu.amount' should be set to Max(1, (numCores / gpuCount)). - 'spark.rapids.sql.concurrentGpuTasks' should be set to Max(4, (gpuMemory / 8G)). - 'spark.rapids.memory.pinnedPool.size' should be set to 2048m. - 'spark.sql.adaptive.enabled' should be enabled for better performance."
app-20220210004538-0189,"Cannot recommend properties. See Comments. Comments: - java.io.FileNotFoundException: File worker_info.yaml does not exist - 'spark.executor.memory' should be set to at least 2GB/core. - 'spark.executor.instances' should be set to (gpuCount * numWorkers). - 'spark.task.resource.gpu.amount' should be set to Max(1, (numCores / gpuCount)). - 'spark.rapids.sql.concurrentGpuTasks' should be set to Max(4, (gpuMemory / 8G)). - 'spark.rapids.memory.pinnedPool.size' should be set to 2048m. - 'spark.sql.adaptive.enabled' should be enabled for better performance."
app-20220210000414-0117,"Cannot recommend properties. See Comments. Comments: - java.io.FileNotFoundException: File worker_info.yaml does not exist - 'spark.executor.memory' should be set to at least 2GB/core. - 'spark.executor.instances' should be set to (gpuCount * numWorkers). - 'spark.task.resource.gpu.amount' should be set to Max(1, (numCores / gpuCount)). - 'spark.rapids.sql.concurrentGpuTasks' should be set to Max(4, (gpuMemory / 8G)). - 'spark.rapids.memory.pinnedPool.size' should be set to 2048m. - 'spark.sql.adaptive.enabled' should be enabled for better performance."
app-20220210005713-0210,"Cannot recommend properties. See Comments. Comments: - java.io.FileNotFoundException: File worker_info.yaml does not exist - 'spark.executor.memory' should be set to at least 2GB/core. - 'spark.executor.instances' should be set to (gpuCount * numWorkers). - 'spark.task.resource.gpu.amount' should be set to Max(1, (numCores / gpuCount)). - 'spark.rapids.sql.concurrentGpuTasks' should be set to Max(4, (gpuMemory / 8G)). - 'spark.rapids.memory.pinnedPool.size' should be set to 2048m. - 'spark.sql.adaptive.enabled' should be enabled for better performance."
app-20220210000744-0123,"Cannot recommend properties. See Comments. Comments: - java.io.FileNotFoundException: File worker_info.yaml does not exist - 'spark.executor.memory' should be set to at least 2GB/core. - 'spark.executor.instances' should be set to (gpuCount * numWorkers). - 'spark.task.resource.gpu.amount' should be set to Max(1, (numCores / gpuCount)). - 'spark.rapids.sql.concurrentGpuTasks' should be set to Max(4, (gpuMemory / 8G)). - 'spark.rapids.memory.pinnedPool.size' should be set to 2048m. - 'spark.sql.adaptive.enabled' should be enabled for better performance."
app-20220210002521-0154,"Cannot recommend properties. See Comments. Comments: - java.io.FileNotFoundException: File worker_info.yaml does not exist - 'spark.executor.memory' should be set to at least 2GB/core. - 'spark.executor.instances' should be set to (gpuCount * numWorkers). - 'spark.task.resource.gpu.amount' should be set to Max(1, (numCores / gpuCount)). - 'spark.rapids.sql.concurrentGpuTasks' should be set to Max(4, (gpuMemory / 8G)). - 'spark.rapids.memory.pinnedPool.size' should be set to 2048m. - 'spark.sql.adaptive.enabled' should be enabled for better performance."
app-20220210004801-0193,"Cannot recommend properties. See Comments. Comments: - java.io.FileNotFoundException: File worker_info.yaml does not exist - 'spark.executor.memory' should be set to at least 2GB/core. - 'spark.executor.instances' should be set to (gpuCount * numWorkers). - 'spark.task.resource.gpu.amount' should be set to Max(1, (numCores / gpuCount)). - 'spark.rapids.sql.concurrentGpuTasks' should be set to Max(4, (gpuMemory / 8G)). - 'spark.rapids.memory.

## Per-App Profile

In [0]:
for x in os.scandir(OUTPUT_DIR + "/rapids_4_spark_profile/"):
  print("APPLICATION ID = " + str(x))
  log = open(x.path + "/profile.log", 'r')
  log_contents = log.read()
  print(log_contents)

APPLICATION ID = <DirEntry 'app-20220210005817-0212'>
### A. Information Collected ###
Application Information:
+--------+------------------------+-----------------------+---------+-------------+-------------+--------+-----------+------------+-------------+
appIndex|appName |appId |sparkUser|startTime |endTime |duration|durationStr|sparkVersion|pluginEnabled|
+--------+------------------------+-----------------------+---------+-------------+-------------+--------+-----------+------------+-------------+
1 |TPC-DS Like Bench ss_max|app-20220210005817-0212|rladmin |1644454696306|1644454724331|28025 |28 s |3.1.1 |true |
+--------+------------------------+-----------------------+---------+-------------+-------------+--------+-----------+------------+-------------+

Data Source Information:
+--------+-----+------------+----------------------------------------------------------------------------------------------------+-------------+---------------------------------------------------------------------------------------------+
appIndex|sqlID|format |location |pushedFilters|schema |
+--------+-----+------------+----------------------------------------------------------------------------------------------------+-------------+---------------------------------------------------------------------------------------------+
1 |24 |Parquet(GPU)|InMemoryFileIndex[hdfs://rl-r7525-d32-u38.raplab.nvidia.com:9000/data/tpcds_sf1000-parquet/useDec...|[] |ss_sold_time_sk:int,ss_item_sk:int,ss_customer_sk:int,ss_cdemo_sk:int,ss_hdemo_sk:int,ss_a...|
+--------+-----+------------+----------------------------------------------------------------------------------------------------+-------------+---------------------------------------------------------------------------------------------+

Executor Information:
+--------+-----------------+------------+-------------+----------+------------+-------------+--------------+------------------+---------------+-------+-------+
appIndex|resourceProfileId|numExecutors|executorCores|maxMem |maxOnHeapMem|maxOffHeapMem|executorMemory|numGpusPerExecutor|executorOffHeap|taskCpu|taskGpu|
+--------+-----------------+------------+-------------+----------+------------+-------------+--------------+------------------+---------------+-------+-------+
1 |0 |8 |16 |8974132838|8974132838 |0 |16384 |1 |0 |1.0 |0.0 |
+--------+-----------------+------------+-------------+----------+------------+-------------+--------------+------------------+---------------+-------+-------+

Job Information:
+--------+-----+----------+-----+-------------+-------------+
appIndex|jobID|stageIds |sqlID|startTime |endTime |
+--------+-----+----------+-----+-------------+-------------+
1 |0 |[0] |null |1644454698958|1644454708288|
1 |1 |[1] |null |1644454708668|1644454708942|
1 |2 |[2] |null |1644454710791|1644454711514|
1 |3 |[3] |null |1644454711725|1644454712154|
1 |4 |[4] |null |1644454712212|1644454712271|
1 |5 |[5] |null |1644454712323|1644454712849|
1 |6 |[6] |null |1644454712915|1644454713390|
1 |7 |[7] |null |1644454713528|1644454713994|
1 |8 |[8] |null |1644454714082|1644454714702|
1 |9 |[9] |null |1644454714837|1644454714865|
1 |10 |[10] |null |1644454714925|1644454715539|
1 |11 |[11] |null |1644454715677|1644454715707|
1 |12 |[12] |null |1644454715774|1644454716298|
1 |13 |[13] |null |1644454716423|1644454716459|
1 |14 |[14] |null |1644454716522|1644454716548|
1 |15 |[15] |null |1644454716598|1644454716625|
1 |16 |[16] |null |1644454716671|1644454716701|
1 |17 |[17] |null |1644454716744|1644454717009|
1 |18 |[18] |null |1644454717046|1644454717067|
1 |19 |[19] |null |1644454717100|1644454717961|
1 |20 |[20] |null |1644454718005|1644454718027|
1 |21 |[21] |null |1644454718061|1644454718662|
1 |22 |[22] |null |1644454718694|1644454719084|
1 |23 |[23] |null |1644454719120|1644454719150|
1 |24 |[24] |null |1644454719184|1644454719208|
1 |25 |[25] |null |1644454719241|1644454719262|
1 |26 |[26] |null |1644454719296|1644454719317|
1 |27 |[27] |null |